### total population

In [8]:
import pandas as pd

df = pd.read_csv(
    "Agregados_por_setores_demografia_BR.csv",
    sep=";",
    dtype=str,
    engine="python"
)

# Filter São Paulo city (3550308 prefix)
df_sp = df[df["CD_setor"].str.startswith("3550308")]

# Keep only needed columns
df_sp = df_sp[["CD_setor", "V01007", "V01008"]]

# Rename
df_sp = df_sp.rename(columns={
    "CD_setor": "tract_id",
    "V01007": "male_population",
    "V01008": "female_population"
})

# Convert numeric
df_sp["male_population"] = pd.to_numeric(df_sp["male_population"], errors="coerce")
df_sp["female_population"] = pd.to_numeric(df_sp["female_population"], errors="coerce")

# Save
df_sp.to_csv("sp_city_male_female_population.csv", index=False)

df_sp.head()


,tract_id,male_population,female_population
324330,355030801000001,329.0,353.0
324331,355030801000002,655.0,719.0
324332,355030801000003,259.0,298.0
324333,355030801000004,263.0,263.0
324334,355030801000005,276.0,303.0


###age

In [2]:
import pandas as pd

df = pd.read_csv(
    "Agregados_por_setores_alfabetizacao_BR.csv",
    sep=";",
    dtype=str,
    engine="python",
    on_bad_lines="skip"
)

# Clean messy IBGE headers
df.columns = df.columns.str.replace('"', '', regex=False).str.strip().str.replace(";", "")

print(df.columns.tolist())  # debug

# Filter SP city
df_sp = df[df["CD_setor"].str.startswith("3550308")]

rename_map = {
    "V00644": "age_15to19",
    "V00645": "age_20to24",
    "V00646": "age_25to29",
    "V00647": "age_30to34",
    "V00648": "age_35to39",
    "V00649": "age_40to44",
    "V00650": "age_45to49",
    "V00651": "age_50to54",
    "V00652": "age_55to59",
    "V00653": "age_60to64",
    "V00654": "age_65to69",
    "V00655": "age_70to79",
    "V00656": "age_80plus"
}

cols = ["CD_setor"] + list(rename_map.keys())

df_age = df_sp[cols].rename(columns=rename_map)
df_age = df_age.rename(columns={"CD_setor": "tract_id"})

df_age.to_csv("sp_city_age_groups_clean.csv", index=False)

df_age.head()


['CD_setor', 'V00644', 'V00645', 'V00646', 'V00647', 'V00648', 'V00649', 'V00650', 'V00651', 'V00652', 'V00653', 'V00654', 'V00655', 'V00656', 'V00657', 'V00658', 'V00659', 'V00660', 'V00661', 'V00662', 'V00663', 'V00664', 'V00665', 'V00666', 'V00667', 'V00668', 'V00669', 'V00670', 'V00671', 'V00672', 'V00673', 'V00674', 'V00675', 'V00676', 'V00677', 'V00678', 'V00679', 'V00680', 'V00681', 'V00682', 'V00683', 'V00684', 'V00685', 'V00686', 'V00687', 'V00688', 'V00689', 'V00690', 'V00691', 'V00692', 'V00693', 'V00694', 'V00695', 'V00696', 'V00697', 'V00698', 'V00699', 'V00700', 'V00701', 'V00702', 'V00703', 'V00704', 'V00705', 'V00706', 'V00707', 'V00708', 'V00709', 'V00710', 'V00711', 'V00712', 'V00713', 'V00714', 'V00715', 'V00716', 'V00717', 'V00718', 'V00719', 'V00720', 'V00721', 'V00722', 'V00723', 'V00724', 'V00725', 'V00726', 'V00727', 'V00728', 'V00729', 'V00730', 'V00731', 'V00732', 'V00733', 'V00734', 'V00735', 'V00736', 'V00737', 'V00738', 'V00739', 'V00740', 'V00741', 'V00742

,tract_id,age_15to19,age_20to24,age_25to29,age_30to34,age_35to39,age_40to44,age_45to49,age_50to54,age_55to59,age_60to64,age_65to69,age_70to79,age_80plus
324330,355030801000001,23,39,43,52,54,63,40,50,60,47,31,62,34
324331,355030801000002,54,96,199,175,149,106,78,76,63,56,48,58,39
324332,355030801000003,35,30,32,31,41,45,50,42,34,44,38,45,33
324333,355030801000004,20,47,39,32,30,36,51,60,35,31,24,44,12
324334,355030801000005,34,37,49,28,38,37,61,59,55,39,25,44,18


###income

In [3]:
import pandas as pd
import re

# Load raw file as text
with open("Agregados_por_setores_renda_responsavel_BR.csv", "r", encoding="latin1") as f:
    lines = f.readlines()

clean_rows = []

for line in lines:
    # remove quotes
    line = line.replace('"', '')

    # split by ";"
    parts = line.strip().split(";")

    fixed_parts = []
    for p in parts:
        # extract all digit groups (handles: "2126   44")
        digits = re.findall(r'\d+', p)

        if len(digits) == 1:
            fixed_parts.append(digits[0])
        elif len(digits) > 1:
            fixed_parts.append("".join(digits))   # concatenate 2126 + 44 → 212644
        else:
            fixed_parts.append("")

    clean_rows.append(fixed_parts)

# Convert to DataFrame
df = pd.DataFrame(clean_rows)

# Assign the correct columns (only what we need)
df.columns = ["tract_id", "V06001", "V06002", "V06003", "V06004", "V06005"]

# Convert V06004 to numeric (others are ignored)
df["V06004"] = pd.to_numeric(df["V06004"], errors="coerce")

# Filter for São Paulo city census tracts
df_sp = df[df["tract_id"].str.startswith("3550308")]

# Select required columns only
df_final = df_sp[["tract_id", "V06004"]].rename(columns={"V06004": "avg_income"})

# Save the clean dataset
df_final.to_csv("sp_city_avg_income.csv", index=False)

df_final.head()
len(df_final)



26889

###race

In [4]:
import pandas as pd
import re

filepath = "Agregados_por_setores_cor_ou_raca_BR.csv"   # <-- put your file name here

# -------------------------------------------
# STEP 1 — Read raw file as lines
# -------------------------------------------
with open(filepath, "r", encoding="latin1") as f:
    raw_lines = f.readlines()

clean_rows = []

for line in raw_lines:
    # Remove spaces, quotes, tabs
    line = line.replace('"', '').replace("\t", "")
    # Split on semicolon
    parts = line.strip().split(";")
    # Remove empty values
    parts = [p.strip() for p in parts if p.strip() != ""]
    clean_rows.append(parts)

# Put into DataFrame with widest row
max_len = max(len(r) for r in clean_rows)
df = pd.DataFrame([r + [""]*(max_len-len(r)) for r in clean_rows])

# -------------------------------------------
# STEP 2 — Rename columns properly
# (Assuming dictionary order matches actual file)
# -------------------------------------------
df = df.rename(columns={
    0: "tract_id",
    1: "V01317",
    2: "V01318",
    3: "V01319",
    4: "V01320",
    5: "V01321"
})

# Keep only required columns
df_race = df[["tract_id","V01317","V01318","V01319","V01320","V01321"]]

# -------------------------------------------
# STEP 3 — Clean values ("X" → NaN, convert numbers)
# -------------------------------------------
df_race = df_race.replace("X", None)

for col in ["V01317","V01318","V01319","V01320","V01321"]:
    df_race[col] = pd.to_numeric(df_race[col], errors="coerce")

# -------------------------------------------
# STEP 4 — Rename race columns
# -------------------------------------------
df_race = df_race.rename(columns={
    "V01317": "race_white",
    "V01318": "race_black",
    "V01319": "race_asian",
    "V01320": "race_brown",
    "V01321": "race_indigenous"
})

# -------------------------------------------
# STEP 5 — Filter São Paulo city tracts
# -------------------------------------------
df_race = df_race[df_race["tract_id"].str.startswith("3550308")]

# -------------------------------------------
# STEP 6 — Save final file
# -------------------------------------------
df_race.to_csv("sp_city_race_groups.csv", index=False)

df_race.head()



,tract_id,race_white,race_black,race_asian,race_brown,race_indigenous
324331,355030801000001,514.0,21.0,3.0,144.0,0.0
324332,355030801000002,1000.0,80.0,9.0,285.0,0.0
324333,355030801000003,428.0,21.0,NaN,106.0,0.0
324334,355030801000004,385.0,15.0,6.0,120.0,0.0
324335,355030801000005,444.0,35.0,9.0,91.0,0.0


###centroids

In [6]:
import zipfile
import geopandas as gpd
import os

# --- unzip ---
zipfile.ZipFile("SP_setores_CD2022.zip").extractall("shp")

# --- load shapefile ---
shp_path = [f for f in os.listdir("shp") if f.endswith(".shp")][0]
gdf = gpd.read_file(f"shp/{shp_path}")

# --- Step 1: convert to UTM zone 23S (meters) ---
gdf_utm = gdf.to_crs(31983)

# --- Step 2: compute centroid in UTM ---
gdf_utm["centroid"] = gdf_utm.geometry.centroid

# --- Step 3: convert centroid back to lat/lon (WGS84) ---
gdf_latlon = gdf_utm.set_geometry("centroid").to_crs(4326)

# --- get actual centroid coordinates ---
gdf["lat"] = gdf_latlon.geometry.y
gdf["lon"] = gdf_latlon.geometry.x

# --- identify tract id column ---
tract_col = "CD_SETOR" if "CD_SETOR" in gdf.columns else gdf.columns[0]

# --- filter São Paulo city only ---
gdf_city = gdf[gdf[tract_col].astype(str).str.startswith("3550308")]

# --- final df ---
df_centroids = gdf_city[[tract_col, "lat", "lon"]].rename(columns={tract_col: "tract_id"})

df_centroids.to_csv("sp_city_tract_centroids.csv", index=False)

df_centroids.head()





,tract_id,lat,lon
66805,355030801000001,-23.567836,-46.570841
66806,355030801000002,-23.566562,-46.568271
66807,355030801000003,-23.568594,-46.567571
66808,355030801000004,-23.570255,-46.568904
66809,355030801000005,-23.571521,-46.570585


### final merged

In [3]:
import pandas as pd

# Load files
age = pd.read_csv("sp_city_age_groups_clean.csv")
income = pd.read_csv("sp_city_avg_income.csv")
mf = pd.read_csv("sp_city_male_female_population.csv")
race = pd.read_csv("sp_city_race_groups.csv")
centroids = pd.read_csv("sp_city_tract_centroids.csv")

# Ensure tract_id is clean
for df in [age, income, mf, race, centroids]:
    df["tract_id"] = df["tract_id"].astype(str).str.strip()

# ---- MERGE (INNER JOIN = intersection only) ----
merged = age.merge(income, on="tract_id", how="inner") \
            .merge(mf, on="tract_id", how="inner") \
            .merge(race, on="tract_id", how="inner") \
            .merge(centroids, on="tract_id", how="inner")

# ---- CLEANING STEP ----
# Replace "X" or empty strings with NaN first
merged = merged.replace(["X", "x", ""], pd.NA)

# Convert all columns except tract_id to numeric
for col in merged.columns:
    if col != "tract_id":
        merged[col] = pd.to_numeric(merged[col], errors="coerce")

# Replace all NaN with 0
merged = merged.fillna(0)

# Save final clean data
merged.to_csv("sp_city_full_merged_clean.csv", index=False)

print("Final merged rows:", len(merged))
merged.head()



Final merged rows: 26889


,tract_id,age_15to19,age_20to24,age_25to29,age_30to34,age_35to39,age_40to44,age_45to49,age_50to54,age_55to59,...,avg_income,male_population,female_population,race_white,race_black,race_asian,race_brown,race_indigenous,lat,lon
0,355030801000001,23.0,39.0,43.0,52.0,54.0,63.0,40.0,50.0,60.0,...,460386.0,329.0,353.0,514.0,21.0,3.0,144.0,0.0,-23.567836,-46.570841
1,355030801000002,54.0,96.0,199.0,175.0,149.0,106.0,78.0,76.0,63.0,...,603876.0,655.0,719.0,1000.0,80.0,9.0,285.0,0.0,-23.566562,-46.568271
2,355030801000003,35.0,30.0,32.0,31.0,41.0,45.0,50.0,42.0,34.0,...,317471.0,259.0,298.0,428.0,21.0,0.0,106.0,0.0,-23.568594,-46.567571
3,355030801000004,20.0,47.0,39.0,32.0,30.0,36.0,51.0,60.0,35.0,...,309764.0,263.0,263.0,385.0,15.0,6.0,120.0,0.0,-23.570255,-46.568904
4,355030801000005,34.0,37.0,49.0,28.0,38.0,37.0,61.0,59.0,55.0,...,300378.0,276.0,303.0,444.0,35.0,9.0,91.0,0.0,-23.571521,-46.570585
